In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
import sys
import numpy as np
from tqdm import tqdm_notebook as tqdm
import torch
import torch.utils.data
import torch.nn as nn
from PIL import Image

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [9]:
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
# import torchvision.datasets as dset

sys.path.append('../maskrcnntools')
import transforms as T
from coco import CocoDetection
from engine import MaskRCNNTrainer
import utils
from logger import Logger

In [10]:
sys.path.append('../lib/coco2voc')
from coco2voc_aux import annsToSeg

In [11]:
# path to COCO
COCO_DATASET = '../../../datasets/COCO/'
TRAIN_DIR = os.path.join(COCO_DATASET, 'train2017')
VAL_DIR = os.path.join(COCO_DATASET, 'val2017')
ANNOTATIONS_DIR = os.path.join(COCO_DATASET, 'annotations')
TRAIN_ANNOT = os.path.join(ANNOTATIONS_DIR, 'instances_train2017.json')
VAL_ANNOT = os.path.join(ANNOTATIONS_DIR, 'instances_val2017.json')

# path to logging dir
TB_LOGS_DIR = '../logs'

In [12]:
coco_train = CocoDetection(root=TRAIN_DIR,
                           annFile=TRAIN_ANNOT)
coco_val = CocoDetection(root=VAL_DIR,
                         annFile=VAL_ANNOT)

loading annotations into memory...
Done (t=14.61s)
creating index...
index created!
loading annotations into memory...
Done (t=0.41s)
creating index...
index created!


In [13]:
def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [51]:
train_ids = np.load('../maskrcnntools/valid_train_ids.npy')
val_ids = np.load('../maskrcnntools/valid_val_ids.npy')

In [52]:
class CocoDataset(object):
    def __init__(self, torch_dataset, valid_ids, transforms=None):
        self.torch_dataset = torch_dataset
        self.transforms = transforms
        self.valid_ids = valid_ids 
        
    def __getitem__(self, idx):
        idx = self.valid_ids[idx]
        img, target = self.torch_dataset[idx]
        
        masks = annsToSeg(target, self.torch_dataset.coco)[3]
        masks = masks.astype(bool)
        obj_ids = [x['category_id'] for x in target]
        
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])
            
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = [x['category_id'] for x in target]
        labels = torch.as_tensor(labels, dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        iscrowd = [x['iscrowd'] for x in target]
        iscrowd = torch.as_tensor(iscrowd, dtype=torch.int64)
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target
    
    def __len__(self):
        return len(self.valid_ids)

In [53]:
train_dataset = CocoDataset(coco_train, train_ids, get_transform(train=True))
val_dataset = CocoDataset(coco_val, val_ids, get_transform(train=False))

In [58]:
# invalid_ids = []
# for i in tqdm(range(len(val_dataset))):
#     try: 
#         sample = val_dataset[i]
#         boxes = sample[1]['boxes']
#         for box in boxes:
#             if box[2] <= box[0] or box[3] <= box[1]:
#                 print(sample[1]['image_id'].numpy()[0])
#                 invalid_ids += [sample[1]['image_id'].numpy()[0]]
#                 break
#     except:
#         invalid_ids += [i]

In [59]:
model = maskrcnn_resnet50_fpn(
            pretrained=False,
            progress=True,
            num_classes=172,
            pretrained_backbone=False)

In [83]:
dataset_train = CocoDataset(coco_train, train_ids, get_transform(train=True))
dataset_val = CocoDataset(coco_val, val_ids, get_transform(train=False))

data_loader_train = torch.utils.data.DataLoader(
    dataset_train, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_val = torch.utils.data.DataLoader(
    dataset_val, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

In [89]:
num_classes = 172

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0001, weight_decay=0.0005)

logger = Logger(TB_LOGS_DIR)

In [90]:
from multiprocessing import Process
import time

def start_tensorboard(command):
    time.sleep(10)
    os.system(command)

In [91]:
print('Monitor here:')
print('\ttensorboard --logdir="{}"'.format(TB_LOGS_DIR))

command = 'tensorboard --logdir="{}"'.format(TB_LOGS_DIR)
p = Process(target=start_tensorboard, args=[command,])
p.start()

Monitor here:
	tensorboard --logdir="../logs"


In [92]:
trainer = MaskRCNNTrainer(model,
                          optimizer,
                          data_loader_train,
                          device, 
                          init_epoch=0, 
                          print_freq=10, 
                          logger=logger, 
                          emergency=False)

In [93]:
num_epochs = 10

for epoch in range(num_epochs):
    trainer.train_one_epoch(lr_schedule=None)
    evaluator = trainer.evaluate()

Epoch: [0]  [    0/58434]  eta: 1 day, 6:44:54  lr: 0.000100  loss: 1.1290 (1.1290)  loss_classifier: 0.2817 (0.2817)  loss_box_reg: 0.0873 (0.0873)  loss_mask: 0.6451 (0.6451)  loss_objectness: 0.1107 (0.1107)  loss_rpn_box_reg: 0.0043 (0.0043)  time: 1.8944  data: 0.4751  max mem: 6937
Epoch: [0]  [   10/58434]  eta: 10:12:40  lr: 0.000100  loss: 1.3426 (1.4899)  loss_classifier: 0.3337 (0.3451)  loss_box_reg: 0.0873 (0.0897)  loss_mask: 0.6244 (0.6154)  loss_objectness: 0.2523 (0.2773)  loss_rpn_box_reg: 0.1590 (0.1624)  time: 0.6292  data: 0.0488  max mem: 8274
Epoch: [0]  [   20/58434]  eta: 9:22:57  lr: 0.000100  loss: 1.5828 (1.5686)  loss_classifier: 0.3305 (0.3823)  loss_box_reg: 0.0758 (0.1062)  loss_mask: 0.6288 (0.6293)  loss_objectness: 0.2523 (0.2729)  loss_rpn_box_reg: 0.1590 (0.1779)  time: 0.5124  data: 0.0075  max mem: 8283
Epoch: [0]  [   30/58434]  eta: 9:07:40  lr: 0.000100  loss: 1.2655 (1.5075)  loss_classifier: 0.2829 (0.3663)  loss_box_reg: 0.0789 (0.1067)  los

Epoch: [0]  [  300/58434]  eta: 8:42:52  lr: 0.000100  loss: 1.6105 (1.6495)  loss_classifier: 0.4928 (0.4311)  loss_box_reg: 0.1416 (0.1322)  loss_mask: 0.6437 (0.6294)  loss_objectness: 0.2408 (0.2850)  loss_rpn_box_reg: 0.1187 (0.1718)  time: 0.5683  data: 0.0070  max mem: 9006
Epoch: [0]  [  310/58434]  eta: 8:42:09  lr: 0.000100  loss: 1.6459 (1.6481)  loss_classifier: 0.5341 (0.4322)  loss_box_reg: 0.1332 (0.1318)  loss_mask: 0.6437 (0.6286)  loss_objectness: 0.1971 (0.2838)  loss_rpn_box_reg: 0.1097 (0.1717)  time: 0.5691  data: 0.0072  max mem: 9006
Epoch: [0]  [  320/58434]  eta: 8:41:58  lr: 0.000100  loss: 1.6867 (1.6567)  loss_classifier: 0.5048 (0.4348)  loss_box_reg: 0.1205 (0.1328)  loss_mask: 0.6171 (0.6287)  loss_objectness: 0.2685 (0.2855)  loss_rpn_box_reg: 0.1097 (0.1748)  time: 0.5277  data: 0.0074  max mem: 9006
Epoch: [0]  [  330/58434]  eta: 8:41:12  lr: 0.000100  loss: 1.7216 (1.6560)  loss_classifier: 0.4539 (0.4343)  loss_box_reg: 0.1357 (0.1327)  loss_mask: 

Epoch: [0]  [  600/58434]  eta: 8:33:05  lr: 0.000100  loss: 1.6053 (1.6479)  loss_classifier: 0.3872 (0.4349)  loss_box_reg: 0.1226 (0.1362)  loss_mask: 0.6160 (0.6199)  loss_objectness: 0.2562 (0.2844)  loss_rpn_box_reg: 0.1482 (0.1725)  time: 0.5425  data: 0.0067  max mem: 9068
Epoch: [0]  [  610/58434]  eta: 8:32:01  lr: 0.000100  loss: 1.6161 (1.6490)  loss_classifier: 0.4043 (0.4359)  loss_box_reg: 0.1367 (0.1365)  loss_mask: 0.6016 (0.6195)  loss_objectness: 0.2901 (0.2846)  loss_rpn_box_reg: 0.1232 (0.1726)  time: 0.5257  data: 0.0065  max mem: 9068
Epoch: [0]  [  620/58434]  eta: 8:31:44  lr: 0.000100  loss: 1.7262 (1.6525)  loss_classifier: 0.4568 (0.4378)  loss_box_reg: 0.1627 (0.1372)  loss_mask: 0.5941 (0.6192)  loss_objectness: 0.2901 (0.2851)  loss_rpn_box_reg: 0.1518 (0.1732)  time: 0.4947  data: 0.0061  max mem: 9068
Epoch: [0]  [  630/58434]  eta: 8:31:20  lr: 0.000100  loss: 1.7146 (1.6536)  loss_classifier: 0.5400 (0.4385)  loss_box_reg: 0.1462 (0.1375)  loss_mask: 

Epoch: [0]  [  900/58434]  eta: 8:28:57  lr: 0.000100  loss: 1.8319 (1.6439)  loss_classifier: 0.4724 (0.4399)  loss_box_reg: 0.1459 (0.1403)  loss_mask: 0.6057 (0.6141)  loss_objectness: 0.3143 (0.2789)  loss_rpn_box_reg: 0.1591 (0.1707)  time: 0.5649  data: 0.0071  max mem: 9078
Epoch: [0]  [  910/58434]  eta: 8:28:44  lr: 0.000100  loss: 1.7589 (1.6436)  loss_classifier: 0.4221 (0.4394)  loss_box_reg: 0.1292 (0.1401)  loss_mask: 0.5962 (0.6137)  loss_objectness: 0.2667 (0.2790)  loss_rpn_box_reg: 0.1516 (0.1714)  time: 0.5695  data: 0.0071  max mem: 9078
Epoch: [0]  [  920/58434]  eta: 8:28:24  lr: 0.000100  loss: 1.4471 (1.6420)  loss_classifier: 0.3563 (0.4391)  loss_box_reg: 0.1040 (0.1401)  loss_mask: 0.6060 (0.6137)  loss_objectness: 0.1976 (0.2784)  loss_rpn_box_reg: 0.0928 (0.1708)  time: 0.5125  data: 0.0070  max mem: 9078
Epoch: [0]  [  930/58434]  eta: 8:28:03  lr: 0.000100  loss: 1.3712 (1.6392)  loss_classifier: 0.3160 (0.4373)  loss_box_reg: 0.0948 (0.1394)  loss_mask: 

Epoch: [0]  [ 1200/58434]  eta: 8:16:45  lr: 0.000100  loss: 1.7252 (1.6257)  loss_classifier: 0.3893 (0.4339)  loss_box_reg: 0.1248 (0.1386)  loss_mask: 0.5976 (0.6099)  loss_objectness: 0.2519 (0.2732)  loss_rpn_box_reg: 0.1788 (0.1701)  time: 0.4983  data: 0.0050  max mem: 9168
Epoch: [0]  [ 1210/58434]  eta: 8:16:07  lr: 0.000100  loss: 1.2430 (1.6229)  loss_classifier: 0.2828 (0.4328)  loss_box_reg: 0.0982 (0.1383)  loss_mask: 0.5957 (0.6098)  loss_objectness: 0.2217 (0.2725)  loss_rpn_box_reg: 0.1313 (0.1695)  time: 0.4992  data: 0.0050  max mem: 9168
Epoch: [0]  [ 1220/58434]  eta: 8:15:44  lr: 0.000100  loss: 1.6620 (1.6270)  loss_classifier: 0.3648 (0.4343)  loss_box_reg: 0.1152 (0.1386)  loss_mask: 0.5926 (0.6097)  loss_objectness: 0.2243 (0.2738)  loss_rpn_box_reg: 0.1210 (0.1707)  time: 0.4670  data: 0.0050  max mem: 9168
Epoch: [0]  [ 1230/58434]  eta: 8:15:04  lr: 0.000100  loss: 1.8028 (1.6269)  loss_classifier: 0.5407 (0.4347)  loss_box_reg: 0.1695 (0.1385)  loss_mask: 

...

Epoch: [0]  [ 8400/58434]  eta: 7:09:44  lr: 0.000100  loss: 1.4873 (1.5537)  loss_classifier: 0.3769 (0.4433)  loss_box_reg: 0.1254 (0.1442)  loss_mask: 0.5192 (0.5645)  loss_objectness: 0.1809 (0.2403)  loss_rpn_box_reg: 0.0889 (0.1614)  time: 0.4932  data: 0.0047  max mem: 9460
Epoch: [0]  [ 8410/58434]  eta: 7:09:37  lr: 0.000100  loss: 1.2968 (1.5536)  loss_classifier: 0.3712 (0.4433)  loss_box_reg: 0.1205 (0.1441)  loss_mask: 0.5207 (0.5645)  loss_objectness: 0.1340 (0.2403)  loss_rpn_box_reg: 0.0714 (0.1614)  time: 0.5132  data: 0.0048  max mem: 9460
Epoch: [0]  [ 8420/58434]  eta: 7:09:29  lr: 0.000100  loss: 1.3340 (1.5536)  loss_classifier: 0.4217 (0.4433)  loss_box_reg: 0.1298 (0.1442)  loss_mask: 0.5192 (0.5644)  loss_objectness: 0.2002 (0.2403)  loss_rpn_box_reg: 0.1295 (0.1614)  time: 0.4743  data: 0.0048  max mem: 9460
Epoch: [0]  [ 8430/58434]  eta: 7:09:20  lr: 0.000100  loss: 1.4358 (1.5534)  loss_classifier: 0.4217 (0.4433)  loss_box_reg: 0.1309 (0.1441)  loss_mask: 

Epoch: [0]  [ 8700/58434]  eta: 7:06:15  lr: 0.000100  loss: 1.3313 (1.5516)  loss_classifier: 0.3811 (0.4433)  loss_box_reg: 0.1307 (0.1440)  loss_mask: 0.5428 (0.5633)  loss_objectness: 0.1552 (0.2396)  loss_rpn_box_reg: 0.0752 (0.1614)  time: 0.5043  data: 0.0052  max mem: 9460
Epoch: [0]  [ 8710/58434]  eta: 7:06:08  lr: 0.000100  loss: 1.2450 (1.5514)  loss_classifier: 0.3737 (0.4432)  loss_box_reg: 0.1277 (0.1440)  loss_mask: 0.5234 (0.5632)  loss_objectness: 0.1159 (0.2396)  loss_rpn_box_reg: 0.0704 (0.1613)  time: 0.5146  data: 0.0049  max mem: 9460
Epoch: [0]  [ 8720/58434]  eta: 7:06:01  lr: 0.000100  loss: 1.2702 (1.5512)  loss_classifier: 0.4273 (0.4433)  loss_box_reg: 0.1588 (0.1441)  loss_mask: 0.4923 (0.5631)  loss_objectness: 0.1214 (0.2395)  loss_rpn_box_reg: 0.0819 (0.1613)  time: 0.4758  data: 0.0047  max mem: 9460
Epoch: [0]  [ 8730/58434]  eta: 7:05:52  lr: 0.000100  loss: 1.2702 (1.5511)  loss_classifier: 0.4444 (0.4433)  loss_box_reg: 0.1585 (0.1441)  loss_mask: 

Epoch: [0]  [ 9000/58434]  eta: 7:03:00  lr: 0.000100  loss: 1.3500 (1.5486)  loss_classifier: 0.4850 (0.4432)  loss_box_reg: 0.1652 (0.1441)  loss_mask: 0.5137 (0.5618)  loss_objectness: 0.1567 (0.2385)  loss_rpn_box_reg: 0.0978 (0.1611)  time: 0.5162  data: 0.0050  max mem: 9460
Epoch: [0]  [ 9010/58434]  eta: 7:02:54  lr: 0.000100  loss: 1.4806 (1.5486)  loss_classifier: 0.4686 (0.4432)  loss_box_reg: 0.1538 (0.1441)  loss_mask: 0.5137 (0.5618)  loss_objectness: 0.1605 (0.2384)  loss_rpn_box_reg: 0.0978 (0.1611)  time: 0.5273  data: 0.0049  max mem: 9460
Epoch: [0]  [ 9020/58434]  eta: 7:02:47  lr: 0.000100  loss: 1.4707 (1.5486)  loss_classifier: 0.4009 (0.4431)  loss_box_reg: 0.1100 (0.1440)  loss_mask: 0.5093 (0.5617)  loss_objectness: 0.2079 (0.2385)  loss_rpn_box_reg: 0.1357 (0.1612)  time: 0.4847  data: 0.0049  max mem: 9460
Epoch: [0]  [ 9030/58434]  eta: 7:02:40  lr: 0.000100  loss: 1.3284 (1.5485)  loss_classifier: 0.3553 (0.4432)  loss_box_reg: 0.0983 (0.1440)  loss_mask: 

Epoch: [0]  [ 9300/58434]  eta: 6:59:50  lr: 0.000100  loss: 1.4712 (1.5474)  loss_classifier: 0.4196 (0.4434)  loss_box_reg: 0.1378 (0.1441)  loss_mask: 0.5152 (0.5604)  loss_objectness: 0.2010 (0.2379)  loss_rpn_box_reg: 0.1416 (0.1616)  time: 0.5269  data: 0.0044  max mem: 9460
Epoch: [0]  [ 9310/58434]  eta: 6:59:43  lr: 0.000100  loss: 1.4045 (1.5475)  loss_classifier: 0.4000 (0.4435)  loss_box_reg: 0.1533 (0.1441)  loss_mask: 0.5289 (0.5603)  loss_objectness: 0.1665 (0.2379)  loss_rpn_box_reg: 0.1087 (0.1617)  time: 0.5338  data: 0.0046  max mem: 9460


KeyboardInterrupt: 

In [ ]:
evaluator = trainer.evaluate()